In [1]:
## plotting individual PAs with treatment & control GEDI shots

In [2]:
install.packages("s3")
#install.packages("doParallel")
#install.packages("RItools")    

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
library("terra")
library("dplyr")
library("sf")
library("s3")
library("sp")
library("foreach")
library("stringr")
library("aws.s3")
#library("doParallel")
#library("RItools")

terra 1.7.29


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.11.2, GDAL 3.7.0, PROJ 9.2.0; sf_use_s2() is TRUE



In [5]:
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
iso3 <- "Bce"
gediwk <- 24
#source(s3_get(paste(f.path,"matching_func.R",sep="")))
source(paste(f.path,"vl_GEDI-PA_2024/matching_func_2024.R",sep=""))

In [7]:
## STEP6: [FIGURE 4B] Calculating per pa summary stats, 1 pa per row, contain shot#/PA---------------------------- 
#cat(paste("Step 6: calculating per pa summary stats for", iso3,"\n"))
gedi_paf <- list.files(paste(f.path,"WDPA_extract/pa_stats_ALL/",sep=""), pattern=".csv", full.names = TRUE)
length(gedi_paf)

[1] 1621

In [33]:
total_shots_in_PA <- c()
total_shots_in_Ctrl <- c()
saveRDS(total_shots_in_PA, paste(f.path,"WDPA_extract/total_shots_in_PA.RDS",sep=""))
saveRDS(total_shots_in_Ctrl, paste(f.path,"WDPA_extract/total_shots_in_Ctrl.RDS",sep=""))

In [ ]:
for(i in 1:length(gedi_paf)){
total_shots_in_PA <- readRDS(paste(f.path,"WDPA_extract/total_shots_in_PA.RDS",sep=""))
total_shots_in_Ctrl <- readRDS(paste(f.path,"WDPA_extract/total_shots_in_Ctrl.RDS",sep=""))
#for(i in 1:2){
    this_paf <- gedi_paf[i]
    print(paste(i, ":", basename(this_paf), sep=" "))
    pa_metrics <- read.csv(this_paf) %>% unique()
    if(length(table(pa_metrics$status))<2) {
    cat(iso3, this_paf, "has 0 protected or treatment \n")
    } else if(table(pa_metrics$status)[1]!=0 && table(pa_metrics$status)[2]!=0) {
        shots_in_PA <- nrow(pa_metrics[pa_metrics$status==1,])
        shots_in_Ctrl <- nrow(pa_metrics[pa_metrics$status==0,])
        total_shots_in_PA <- c(total_shots_in_PA, shots_in_PA)
        total_shots_in_Ctrl <- c(total_shots_in_Ctrl, shots_in_Ctrl)
      }
    rm(pa_metrics)
    saveRDS(total_shots_in_PA, paste(f.path,"WDPA_extract/total_shots_in_PA.RDS",sep=""))
    saveRDS(total_shots_in_Ctrl, paste(f.path,"WDPA_extract/total_shots_in_Ctrl.RDS",sep=""))
}
cat("Done counting GEDI shots", "\n")
cat("Total shots in PA: ",sum(total_shots_in_PA),"\n")
cat("Total shots in Ctrl: ",sum(total_shots_in_Ctrl),"\n")


In [ ]:
#for(i in 1:length(gedi_paf)){
for(i in 1:2){
    this_paf <- gedi_paf[i]
    print(basename(this_paf))
    pa_metrics <- read.csv(this_paf) %>% unique()
    if(length(table(pa_metrics$status))<2) {
    cat(iso3, this_paf, "has 0 protected or treatment \n")
    } else if(table(pa_metrics$status)[1]!=0 && table(pa_metrics$status)[2]!=0) {
#--------
    lon_0 <- pa_metrics[pa_metrics$status==0,"lon_lowestmode"]
    lat_0 <- pa_metrics[pa_metrics$status==0,"lat_lowestmode"]
    lon_1 <- pa_metrics[pa_metrics$status==1,"lon_lowestmode"]
    lat_1 <- pa_metrics[pa_metrics$status==1,"lat_lowestmode"]
    
    plot(vect(cbind(lon_0,lat_0), type="points", atts=NULL), pch=".", main=unique(pa_metrics$pa_id))
    points(vect(cbind(lon_1,lat_1), type="points", atts=NULL), pch=".", col="red")
#--------
    }
}      
